In [15]:
!pip install gradio huggingface_hub PyMuPDF sentence-transformers faiss-cpu

In [ ]:
import gradio as gr
from huggingface_hub import InferenceClient
from typing import List, Tuple
import fitz  # PyMuPDF
from sentence_transformers import SentenceTransformer, util
import numpy as np
import faiss

client = InferenceClient("HuggingFaceH4/zephyr-7b-beta", token="HUGGINGFACE_READ_API")

In [45]:
class MyApp:
    def __init__(self) -> None:
        self.documents = []
        self.embeddings = None
        self.index = None
        self.load_pdf("2022.findings-naacl.130.pdf")
        self.build_vector_db()

    def load_pdf(self, file_path: str) -> None:
        """Extracts text from a PDF file and stores it in the app's documents."""
        doc = fitz.open(file_path)
        self.documents = []
        for page_num in range(len(doc)):
            page = doc[page_num]
            text = page.get_text()
            self.documents.append({"page": page_num + 1, "content": text})
        print("PDF processed successfully!")

    def build_vector_db(self) -> None:
        """Builds a vector database using the content of the PDF."""
        model = SentenceTransformer('all-MiniLM-L6-v2')
        # Generate embeddings for all document contents
        self.embeddings = model.encode([doc["content"] for doc in self.documents])
        # Create a FAISS index
        self.index = faiss.IndexFlatL2(self.embeddings.shape[1])
        # Add the embeddings to the index
        self.index.add(np.array(self.embeddings))
        print("Vector database built successfully!")
        
    def search_documents(self, query: str, k: int = 3) -> List[str]:
        """Searches for relevant documents using vector similarity."""
        model = SentenceTransformer('all-MiniLM-L6-v2')
        # Generate an embedding for the query
        query_embedding = model.encode([query])
        # Perform a search in the FAISS index
        D, I = self.index.search(np.array(query_embedding), k)
        # Retrieve the top-k documents
        results = [self.documents[i]["content"] for i in I[0]]
        return results if results else ["No relevant documents found."]

In [46]:
app = MyApp()

def respond(
    message: str,
    history: List[Tuple[str, str]],
    system_message: str,
    max_tokens: int,
    temperature: float,
    top_p: float,
):
    system_message = "You are an expert in identifying and mitigating spurious correlations in NLP models. " \
                     "You guide users through the document and answer questions based on its content. " \
                     "Please keep your responses concise and to the point."
    messages = [{"role": "system", "content": system_message}]

    for val in history:
        if val[0]:
            messages.append({"role": "user", "content": val[0]})
        if val[1]:
            messages.append({"role": "assistant", "content": val[1]})

    messages.append({"role": "user", "content": message})

    retrieved_docs = app.search_documents(message)
    context = "\n".join(retrieved_docs)
    messages.append({"role": "system", "content": "Relevant documents: " + context})

    response = ""

    for message in client.chat_completion(
        messages,
        max_tokens=max_tokens,
        stream=True,
        temperature=temperature,
        top_p=top_p,
    ):
        token = message.choices[0].delta.content
        response += token
        yield response


PDF processed successfully!
Vector database built successfully!


In [47]:
import gradio as gr

# Assuming the MyApp class and respond function are already defined
# You should have loaded the document "Identifying and Mitigating Spurious Correlations"
# and built the vector database using the upload_document method as described before.

demo = gr.Blocks()

with demo:
    gr.Markdown("📚 **Identifying and Mitigating Spurious Correlations for Improving Robustness in NLP Models**")
    gr.Markdown(
        "‼️ Disclaimer: This chatbot is based on the paper 'Identifying and Mitigating Spurious Correlations for Improving Robustness in NLP Models'. "
        "The responses provided are based on the document's content and do not constitute professional advice. ‼️"
    )
    
    chatbot = gr.ChatInterface(
        respond,  # Assuming the respond function uses the MyApp class and document context
        examples=[
            ["What are spurious correlations in NLP models?"],
            ["How can we mitigate spurious correlations in NLP models?"],
            ["What are the effects of spurious correlations on NLP model robustness?"],
            ["Can you explain the concept of robustness in NLP?"],
            ["What are the key challenges in identifying spurious correlations in NLP?"],
            ["How does dataset design impact spurious correlations in NLP models?"],
            ["What methods can be used to detect spurious correlations?"]
        ],
        title='Spurious Correlations in NLP Models Assistant 📄'
    )

if __name__ == "__main__":
    demo.launch()


c:\Users\ambro\OneDrive\Bureau\RAG\.venv\Lib\site-packages\gradio\components\chatbot.py:255: UserWarning: The 'tuples' format for chatbot messages is deprecated and will be removed in a future version of Gradio. Please set type='messages' instead, which uses openai-style 'role' and 'content' keys.
  warnings.warn(


* Running on local URL:  http://127.0.0.1:7868

To create a public link, set `share=True` in `launch()`.


c:\Users\ambro\OneDrive\Bureau\RAG\.venv\Lib\site-packages\gradio\helpers.py:968: UserWarning: Unexpected argument. Filling with None.
  warnings.warn("Unexpected argument. Filling with None.")
